In [2]:
import os
from flask import Flask, flash, jsonify, redirect, request, send_from_directory
from werkzeug.utils import secure_filename
from ultralytics import YOLO
from PIL import Image, ImageDraw
from flask_cors import CORS
import time

UPLOAD_FOLDER = './store'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

app = Flask(__name__)

CORS(app, resources={r"/*": {"origins": "*"}})

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

model = YOLO('./best.pt')

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    print('Received a request')
    if request.method == 'POST':
       
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
       
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            results = model.predict(os.path.join(app.config['UPLOAD_FOLDER'], filename), conf = 0.2, save_dir = "./output/")
            names = model.names
            store = set()
            for r in results[0]:
                for c in r.boxes.cls:
                    store.add(names[int(c)])
            names_list = list(store)
            
            
            image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            image = Image.open(image_path)
            classes = {0:"Sini", 1:"Jebena"}
            # Draw bounding boxes and labels of detections
            draw = ImageDraw.Draw(image)

            for box_info in results[0].boxes:
                
                x_min, y_min, x_max, y_max = box_info.xyxy.squeeze().tolist()  
                class_id = int(box_info.cls.item())  
                confidence = float(box_info.conf.item()) 
               
                box_color = 'red' if classes[class_id] == 'Jebena' else 'green'
                label = f'{classes[class_id]}'

               
                draw.rectangle([(x_min, y_min), (x_max, y_max)], outline=box_color, width = 6)
                # font =  ImageFont.truetype("arial.ttf", 15)
                draw.text((x_min, y_min), label, fill=box_color, )

            
            image.show()  
            image.save(os.path.join("/home/geleta/Desktop/PAI_ASSIGNMENT/pai_project/assets", filename))
            
            # time.sleep(5)
            return jsonify({'names': names_list, 'image_path': filename})

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

@app.route('/get_image/<filename>', methods=['GET'])
def get_image(filename):
    return send_from_directory('/home/geleta/Desktop/PAI_ASSIGNMENT/pai_project/assets', filename)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.133.126:5000
Press CTRL+C to quit


Received a request

image 1/1 /home/geleta/Desktop/Ethiopic-Object-Detection-App/Backend/store/scaled_ac3b724e-4c07-4f34-bbd1-2e39e07f7d603089627478048943687.jpg_rotated_360.jpg: 640x288 1 Jebena, 430.4ms
Speed: 38.8ms preprocess, 430.4ms inference, 29.8ms postprocess per image at shape (1, 3, 640, 288)


192.168.133.113 - - [28/Jan/2024 10:19:58] "POST / HTTP/1.1" 200 -
192.168.133.113 - - [28/Jan/2024 10:19:58] "GET /get_image/scaled_ac3b724e-4c07-4f34-bbd1-2e39e07f7d603089627478048943687.jpg_rotated_360.jpg HTTP/1.1" 200 -
